In [4]:
import os


root_path = '/media/ext_disk/zhenfang/davis/results/model1/without_cross/'
sub_path = '2/'
model_file = 'full_model_2_50.pt'

#results_save_file='/media/ext_disk/zhenfang/davis/results/model1/without_cross/2/results.txt'
results_file='results.txt'



sub_path=str(int(sub_path[:-1])+1)+'/'
        
model_path=os.path.join(root_path, sub_path, model_file)
results_save_file=os.path.join(root_path,sub_path,results_file)

#results_save_file=os.path.join(root_path,sub_path,model_file)


print(model_path)
print(results_save_file)

/media/ext_disk/zhenfang/davis/results/model1/without_cross/3/full_model_2_50.pt
/media/ext_disk/zhenfang/davis/results/model1/without_cross/3/results.txt


In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
import torchmetrics
import torch.nn as nn

In [2]:
class ConvAttMLP_10layers_add__(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0 = nn.BatchNorm1d(3300)
        self.fc1 = nn.Linear(3300, 1100)
        self.fc11=nn.Linear(1100,1100)
        self.bn1 = nn.BatchNorm1d(1100)
        self.fc2 = nn.Linear(1100, 100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 9)
        self.bn3 = nn.BatchNorm1d(9)

        self.drop = nn.Dropout(0.5)
        
        
        # Conv layers
        self.conv1 = nn.Conv1d(1, 8, 5, padding=2)
        self.conv2 = nn.Conv1d(8, 16, 5, padding=2)
        self.conv3 = nn.Conv1d(16, 16, 5, padding=2)  # Change to match input channel 16
        self.conv4 = nn.Conv1d(16, 1, 1, stride=1)
        #self.conv1 = nn.Conv1d(1, 8, 5, padding=2)
        self.bn4 = nn.BatchNorm1d(3300)
        #self.conv2 = nn.Conv1d(8, 16, 5, padding=2)
        self.bn5 = nn.BatchNorm1d(3300)
        self.pool2 = nn.MaxPool1d(3, stride=3)
        #self.conv3 = nn.Conv1d(16, 1, 1, stride=1)
        self.pool3 = nn.MaxPool1d(3, stride=3)
        self.bn6 = nn.BatchNorm1d(1100)
        
        self.bn7=nn.BatchNorm1d(1100)

        self.avg_pool = nn.AvgPool1d(kernel_size=3300)
        self.max_pool = nn.MaxPool1d(kernel_size=3300)
        self.fc = nn.Sequential(
            nn.Conv1d(16, 16 // 8, 1, padding=0, bias=False),
            nn.ReLU(),
            nn.Conv1d(16 // 8, 16, 1, padding=0, bias=False),
            nn.Sigmoid()
        )

        # Define a list to hold multiple layers
        self.layers = nn.ModuleList([self._create_layer() for _ in range(2)])

    def _create_layer(self):
        """Create a single layer using conv and pooling operations as defined in the original forward method."""
        return nn.Sequential(
            nn.Conv1d(1, 8, 5, padding=2),
            nn.BatchNorm1d(3300),
            nn.Conv1d(8, 16, 5, padding=2),
            nn.Conv1d(16, 16, 5, padding=2),  # Match input/output channel
            nn.Conv1d(16, 1, 1, stride=1),  # This will receive 16 channels and output 1
            nn.AvgPool1d(kernel_size=3300),
            nn.MaxPool1d(kernel_size=3300)
        )

    def forward(self, x):
        x = self.bn0(x)
        x1= F.relu(self.drop(self.bn1(self.fc11(self.fc1(x)))))
        
        
        x0 = torch.unsqueeze(x, 1)  # Add a channel dimension
        #print('x0:',x0.shape)
        # Pass through each layer sequentially
        for layer in self.layers:
            
            x2 = F.relu(layer[0](x0))  # Conv1
            x2 = F.relu(layer[2](x2))  # Conv2
            avg_pool = self.avg_pool(x2)
            avg_out = self.fc(self.avg_pool(x2))
            max_pool = self.max_pool(x2)
            max_out = self.fc(self.max_pool(x2))
            out = avg_out + max_out
            x2 = torch.mul(x2, out)
            x2 = F.relu(layer[4](x2))  # Conv3
            x2 = x0 + x2  # Skip connection
            x0 = x2  # Pass to next layer
            #print('x0+++:',x0.shape)
            
        x2=self.pool3(x2)
        #print('x2:',x2.shape)
        x2=torch.squeeze(x2,1)
        x2=self.bn6(x2)
        #x3=torch.add(x1,x2)
        #x3=torch.concat([x1,x2],dim=1)
        #print('x2_final:',x2.shape)
        x3=torch.add(x1,x2)
        #x3=torch.cat([x1,x2],dim=1)
        x4=self.bn7(x3)
        x4 = F.relu(self.drop(self.bn2(self.fc2(x4))))
        return F.softmax(self.bn3(self.fc3(x4)), dim=1) 
    
#modela=ConvAttMLP_10layers_add__()
#modela.cuda()
#print(modela)    

In [9]:
class ConvAttMLP_10layers_add__LRP(nn.Module):
    def __init__(self):
        super().__init__()
        #self.bn0 = nn.BatchNorm1d(3300)
        self.fc1 = nn.Linear(3300, 1100)
        self.fc11=nn.Linear(1100,1100)
        #self.bn1 = nn.BatchNorm1d(1100)
        self.fc2 = nn.Linear(1100, 100)
        #self.bn2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 9)
        #self.bn3 = nn.BatchNorm1d(9)

        self.drop = nn.Dropout(0.5)
        
        
        # Conv layers
        self.conv1 = nn.Conv1d(1, 8, 5, padding=2)
        self.conv2 = nn.Conv1d(8, 16, 5, padding=2)
        self.conv3 = nn.Conv1d(16, 16, 5, padding=2)  # Change to match input channel 16
        self.conv4 = nn.Conv1d(16, 1, 1, stride=1)
        #self.conv1 = nn.Conv1d(1, 8, 5, padding=2)
        #self.bn4 = nn.BatchNorm1d(3300)
        #self.conv2 = nn.Conv1d(8, 16, 5, padding=2)
        #self.bn5 = nn.BatchNorm1d(3300)
        self.pool2 = nn.MaxPool1d(3, stride=3)
        #self.conv3 = nn.Conv1d(16, 1, 1, stride=1)
        self.pool3 = nn.MaxPool1d(3, stride=3)
        #self.bn6 = nn.BatchNorm1d(1100)
        
        #self.bn7=nn.BatchNorm1d(1100)

        self.avg_pool = nn.AvgPool1d(kernel_size=3300)
        self.max_pool = nn.MaxPool1d(kernel_size=3300)
        self.fc = nn.Sequential(
            nn.Conv1d(16, 16 // 8, 1, padding=0, bias=False),
            nn.ReLU(),
            nn.Conv1d(16 // 8, 16, 1, padding=0, bias=False),
            nn.Sigmoid()
        )

        # Define a list to hold multiple layers
        self.layers = nn.ModuleList([self._create_layer() for _ in range(2)])

    def _create_layer(self):
        """Create a single layer using conv and pooling operations as defined in the original forward method."""
        return nn.Sequential(
            nn.Conv1d(1, 8, 5, padding=2),
            nn.BatchNorm1d(3300),
            nn.Conv1d(8, 16, 5, padding=2),
            nn.Conv1d(16, 16, 5, padding=2),  # Match input/output channel
            nn.Conv1d(16, 1, 1, stride=1),  # This will receive 16 channels and output 1
            nn.AvgPool1d(kernel_size=3300),
            nn.MaxPool1d(kernel_size=3300)
        )

    def forward(self, x):
        #x = self.bn0(x)
        x1= F.relu(self.drop(self.bn1(self.fc11(self.fc1(x)))))
        
        '''
        x0 = torch.unsqueeze(x, 1)  # Add a channel dimension
        #print('x0:',x0.shape)
        # Pass through each layer sequentially
        for layer in self.layers:
            
            x2 = F.relu(layer[0](x0))  # Conv1
            x2 = F.relu(layer[2](x2))  # Conv2
            avg_pool = self.avg_pool(x2)
            avg_out = self.fc(self.avg_pool(x2))
            max_pool = self.max_pool(x2)
            max_out = self.fc(self.max_pool(x2))
            out = avg_out + max_out
            x2 = torch.mul(x2, out)
            x2 = F.relu(layer[4](x2))  # Conv3
            x2 = x0 + x2  # Skip connection
            x0 = x2  # Pass to next layer
            #print('x0+++:',x0.shape)
           
        x2=self.pool3(x2)
        #print('x2:',x2.shape)
        x2=torch.squeeze(x2,1)
        x2=self.bn6(x2)
        #x3=torch.add(x1,x2)
        #x3=torch.concat([x1,x2],dim=1)
        #print('x2_final:',x2.shape)
        '''
        x3=torch.add(x1,x2)
        #x3=torch.cat([x1,x2],dim=1)
        x4=self.bn7(x3)
        x4 = F.relu(self.drop(self.bn2(self.fc2(x4))))
        return F.softmax(self.bn3(self.fc3(x4)), dim=1) 
    
#modela=ConvAttMLP_10layers_add__()
#modela.cuda()
#print(modela)    

In [2]:
class ConvAttMLP_10layers_add(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0 = nn.BatchNorm1d(3300)
        self.fc1 = nn.Linear(3300, 1100)
        self.fc11=nn.Linear(1100,1100)
        self.bn1 = nn.BatchNorm1d(1100)
        self.fc2 = nn.Linear(2200, 100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 9)
        self.bn3 = nn.BatchNorm1d(9)

        self.drop = nn.Dropout(0.5)
        
        
        # Conv layers
        self.conv1 = nn.Conv1d(1, 8, 5, padding=2)
        self.conv2 = nn.Conv1d(8, 16, 5, padding=2)
        self.conv3 = nn.Conv1d(16, 16, 5, padding=2)  # Change to match input channel 16
        self.conv4 = nn.Conv1d(16, 1, 1, stride=1)
        #self.conv1 = nn.Conv1d(1, 8, 5, padding=2)
        self.bn4 = nn.BatchNorm1d(3300)
        #self.conv2 = nn.Conv1d(8, 16, 5, padding=2)
        self.bn5 = nn.BatchNorm1d(3300)
        self.pool2 = nn.MaxPool1d(3, stride=3)
        #self.conv3 = nn.Conv1d(16, 1, 1, stride=1)
        self.pool3 = nn.MaxPool1d(3, stride=3)
        self.bn6 = nn.BatchNorm1d(1100)
        
        self.bn7=nn.BatchNorm1d(2200)

        self.avg_pool = nn.AvgPool1d(kernel_size=3300)
        self.max_pool = nn.MaxPool1d(kernel_size=3300)
        self.fc = nn.Sequential(
            nn.Conv1d(16, 16 // 8, 1, padding=0, bias=False),
            nn.ReLU(),
            nn.Conv1d(16 // 8, 16, 1, padding=0, bias=False),
            nn.Sigmoid()
        )

        # Define a list to hold multiple layers
        self.layers = nn.ModuleList([self._create_layer() for _ in range(2)])

    def _create_layer(self):
        """Create a single layer using conv and pooling operations as defined in the original forward method."""
        return nn.Sequential(
            nn.Conv1d(1, 8, 5, padding=2),
            nn.BatchNorm1d(3300),
            nn.Conv1d(8, 16, 5, padding=2),
            nn.Conv1d(16, 16, 5, padding=2),  # Match input/output channel
            nn.Conv1d(16, 1, 1, stride=1),  # This will receive 16 channels and output 1
            nn.AvgPool1d(kernel_size=3300),
            nn.MaxPool1d(kernel_size=3300)
        )

    def forward(self, x):
        x = self.bn0(x)
        x1= F.relu(self.drop(self.bn1(self.fc11(self.fc1(x)))))
        
        
        x0 = torch.unsqueeze(x, 1)  # Add a channel dimension
        #print('x0:',x0.shape)
        # Pass through each layer sequentially
        for layer in self.layers:
            
            x2 = F.relu(layer[0](x0))  # Conv1
            x2 = F.relu(layer[2](x2))  # Conv2
            avg_pool = self.avg_pool(x2)
            avg_out = self.fc(self.avg_pool(x2))
            max_pool = self.max_pool(x2)
            max_out = self.fc(self.max_pool(x2))
            out = avg_out + max_out
            x2 = torch.mul(x2, out)
            x2 = F.relu(layer[4](x2))  # Conv3
            x2 = x0 + x2  # Skip connection
            x0 = x2  # Pass to next layer
            #print('x0+++:',x0.shape)
            
        x2=self.pool3(x2)
        #print('x2:',x2.shape)
        x2=torch.squeeze(x2,1)
        x2=self.bn6(x2)
        #x3=torch.add(x1,x2)
        #x3=torch.concat([x1,x2],dim=1)
        #print('x2_final:',x2.shape)
        #x3=torch.add(x1,x2)
        x3=torch.cat([x1,x2],dim=1)
        x4=self.bn7(x3)
        x4 = F.relu(self.drop(self.bn2(self.fc2(x4))))
        return F.softmax(self.bn3(self.fc3(x4)), dim=1) 
    
modela=ConvAttMLP_10layers_add()
modela.cuda()
print(modela)    

ConvAttMLP_10layers_add(
  (bn0): BatchNorm1d(3300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3300, out_features=1100, bias=True)
  (fc11): Linear(in_features=1100, out_features=1100, bias=True)
  (bn1): BatchNorm1d(1100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=2200, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=100, out_features=9, bias=True)
  (bn3): BatchNorm1d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
  (conv1): Conv1d(1, 8, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv2): Conv1d(8, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv3): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv4): Conv1d(16, 1, kernel_size=(1,), stride=(1,))
  (bn4): BatchNorm1d(3300, eps=1e-05, momentum=0.1, affin

In [3]:
class ConvAttMLP_10layers_add_LRP(nn.Module):
    def __init__(self):
        super().__init__()
        #self.bn0 = nn.BatchNorm1d(3300)
        self.fc1 = nn.Linear(3300, 1100)
        self.fc11=nn.Linear(1100,1100)
        #self.bn1 = nn.BatchNorm1d(1100)
        #self.fc2 = nn.Linear(2200, 100)
        self.fc22=nn.Linear(1100,100)
        #self.bn2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 9)
        #self.bn3 = nn.BatchNorm1d(9)

        self.drop = nn.Dropout(0.5)
        
        
        # Conv layers
        #self.conv1 = nn.Conv1d(1, 8, 5, padding=2)
        #self.conv2 = nn.Conv1d(8, 16, 5, padding=2)
        #self.conv3 = nn.Conv1d(16, 16, 5, padding=2)  # Change to match input channel 16
        #self.conv4 = nn.Conv1d(16, 1, 1, stride=1)
        #self.conv1 = nn.Conv1d(1, 8, 5, padding=2)
        #self.bn4 = nn.BatchNorm1d(3300)
        #self.conv2 = nn.Conv1d(8, 16, 5, padding=2)
        #self.bn5 = nn.BatchNorm1d(3300)
        #self.pool2 = nn.MaxPool1d(3, stride=3)
        #self.conv3 = nn.Conv1d(16, 1, 1, stride=1)
        #self.pool3 = nn.MaxPool1d(3, stride=3)
        #self.bn6 = nn.BatchNorm1d(1100)
        
        #self.bn7=nn.BatchNorm1d(2200)
        '''
        self.avg_pool = nn.AvgPool1d(kernel_size=3300)
        self.max_pool = nn.MaxPool1d(kernel_size=3300)
        self.fc = nn.Sequential(
            nn.Conv1d(16, 16 // 8, 1, padding=0, bias=False),
            nn.ReLU(),
            nn.Conv1d(16 // 8, 16, 1, padding=0, bias=False),
            nn.Sigmoid()
        )
        
        # Define a list to hold multiple layers
        self.layers = nn.ModuleList([self._create_layer() for _ in range(2)])
        '''
        
    ''' 
    def _create_layer(self):
        """Create a single layer using conv and pooling operations as defined in the original forward method."""
        return nn.Sequential(
            nn.Conv1d(1, 8, 5, padding=2),
            nn.BatchNorm1d(3300),
            nn.Conv1d(8, 16, 5, padding=2),
            nn.Conv1d(16, 16, 5, padding=2),  # Match input/output channel
            nn.Conv1d(16, 1, 1, stride=1),  # This will receive 16 channels and output 1
            nn.AvgPool1d(kernel_size=3300),
            nn.MaxPool1d(kernel_size=3300)
        )
    '''
    def forward(self, x):
        print('x:',x.shape)
        #x2=x[0,3300:]
        #x = self.bn0(x)
        #print('x_bn0:',x.shape)
        #x2=torch.zeros([1,1100])
        #x2=x2.cuda()
        #xm=torch.cat([x,x2],dim=1)
        
        #print('xm.shape:',xm.shape)
        x1= F.relu(self.drop(self.fc11(self.fc1(x))))
        
        """
        x0 = torch.unsqueeze(x, 1)  # Add a channel dimension
        #print('x0:',x0.shape)
        # Pass through each layer sequentially
        for layer in self.layers:
            ）
            x2 = F.relu(layer[0](x0))  # Conv1
            x2 = F.relu(layer[2](x2))  # Conv2
            avg_pool = self.avg_pool(x2)
            avg_out = self.fc(self.avg_pool(x2))
            max_pool = self.max_pool(x2)
            max_out = self.fc(self.max_pool(x2))
            out = avg_out + max_out
            x2 = torch.mul(x2, out)
            x2 = F.relu(layer[4](x2))  # Conv3
            x2 = x0 + x2  # Skip connection
            x0 = x2  # Pass to next layer
            #print('x0+++:',x0.shape)
            
        x2=self.pool3(x2)
        #print('x2:',x2.shape)
        x2=torch.squeeze(x2,1)
        x2=self.bn6(x2)
        #x3=torch.add(x1,x2)
        #x3=torch.concat([x1,x2],dim=1)
        #print('x2_final:',x2.shape)
        #x3=torch.add(x1,x2)
        """
        #print('x1.device:',x1.device)
        #x2=torch.zeros([1,1100])
        #x2=x2.cuda()
        #print('x2.device:',x2.device)
        #print('x1.shape:',x1.shape)
        #print('x2.shape:',x2.shape)
        #x2=torch.squeeze(x2,0)
        #print('x2.shape:',x2.shape)
        #x3=torch.cat([x1,x2],dim=-1)
        #print('x3.shape:',x3.shape)
        #x4=self.bn7(x3)
        x4 = F.relu(self.drop(self.fc22(x1)))
        return F.softmax(self.fc3(x4), dim=-1) 
    
modela=ConvAttMLP_10layers_add_LRP()
modela.cuda()
print(modela)    

ConvAttMLP_10layers_add_LRP(
  (fc1): Linear(in_features=3300, out_features=1100, bias=True)
  (fc11): Linear(in_features=1100, out_features=1100, bias=True)
  (fc22): Linear(in_features=1100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=9, bias=True)
  (drop): Dropout(p=0.5, inplace=False)
)


In [4]:
class ConvAttMLP_LRP(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(3300)
        self.fc1 = nn.Linear(3300, 1100)
        self.bn1= nn.BatchNorm1d(1100)
        self.fc2 = nn.Linear(1100, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.5)
        
        self.conv1 = nn.Conv1d(1, 8, 5, padding=2)
        #self.pool1 = nn.MaxPool1d(3,stride=3)
        self.bn4=nn.BatchNorm1d(3300)
        self.conv2 = nn.Conv1d(8, 16, 5,padding=2)
        self.bn5=nn.BatchNorm1d(3300)
        self.pool2 = nn.MaxPool1d(3,stride=3)
        self.conv3=nn.Conv1d(16,1,1,stride=1)
        self.pool3=nn.MaxPool1d(3,stride=3)
        self.bn6=nn.BatchNorm1d(1100)
        #self.fc1 = nn.Linear(16*5*5, 120)
        #self.fc2 = nn.Linear(120, 84)
        #self.fc3 = nn.Linear(84, 10)
        self.bn7=nn.BatchNorm1d(1100)
        
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.max_pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Sequential(
            nn.Conv1d(16, 16 // 8, 1, padding=0, bias=False),
            nn.ReLU(),
            nn.Conv1d(16 // 8, 16, 1, padding=0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        #x=self.bn0(x)
        #print('x:',x.shape)
        x1= F.relu(self.drop(self.fc1(x)))
        
        '''
        x0=torch.unsqueeze(x,1)
        print("x0:",x0.shape)
        
        
        x2=F.relu(self.conv1(x0))
        #x2=x2+x0
        #print('x2:',x2.shape)
        x2=F.relu(self.conv2(x2))
        #x2=x2+x0
        #print('x2:',x2.shape)
        
        
        
        avg_out = self.fc(self.avg_pool(x2))
        max_out = self.fc(self.max_pool(x2))
        out = avg_out + max_out
        x2=torch.mul(x2,out)
        x2=F.relu(self.conv3(x2))
        print('x2:',x2.shape)
        #x2=x0+x2
        
        #print('x2:',x2.shape)
        #x2=x0+x2####skip connection
        #x2=torch.squeeze(x2,1)
        x2=self.bn5(x2)
        #x2 = torch.unsqueeze(x2, 1)  # 在第 1 维（通道维）添加维度#####################################
        #x2 = self.pool3(x2)  # 现在可以通过 MaxPool1d 了
        x2=self.pool3(x2)
        #print('x2:',x2.shape)
        x2=self.bn6(x2)
        #x3=torch.add(x1,x2)
        #x3=torch.concat([x1,x2],dim=1)
        '''
        x2=torch.zeros([1,1100])
        x3=torch.add(x1,x2)
        #x4=self.bn7(x3)
        x4 = F.relu(self.drop(self.fc2(x3)))
        return F.softmax(self.fc3(x4), dim=1) 


In [5]:
import torch
#import Trainer
#from network import NFM
import torch.utils.data as Data
#from Utils.criteo_loader import getTestData, getTrainData
from torch.nn import functional as F
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
#import torchmetrics
nfm_config = \
{
    'n_class':9,
    'linear_hidden1':2000,
    #'linear_hidden':100,#线性模型输出层（隐层个数）
    #'embed_input_dim':1001,#embed输入维度
    #'embed_dim': 100, # 用于控制稀疏特征经过Embedding层后的稠密特征大小，embed输出维度
    #'dnn_hidden_units': [100,11],#MLP隐层和输出层
    
    'dnn_hidden_units':[100,9],#MLP隐层
    'num_sparse_features_cols':10477,#the number of the gene columns
    'num_dense_features': 0,#dense features number
    'bi_dropout': 0.5,#Bi-Interaction 的dropout
    'num_epoch': 500,#训练epoch次数
    'batch_size': 16,#batch_size
    'lr': 1e-3,
    'l2_regularization': 1e-4,
    'device_id': 0,
    'use_cuda': False,
    'epoch':1000,
    
    #'train_file': '../Data/criteo/processed_data/train_set.csv',
    #'fea_file': '../Data/criteo/processed_data/fea_col.npy',
    #'validate_file': '../Data/criteo/processed_data/val_set.csv',
    #'test_file': '../Data/criteo/processed_data/test_set.csv',
    #'model_name': '../TrainedModels/NFM.model'
    #'train_file':'data/xiaoqiu_gene_5000/train/final_5000_encode_100x.csv',
    #'train_data':'dataset/qiuguan/encode/encode_1000/train/train_encode_data_1000_new.csv',
    #'train_label':'dataset/qiuguan/non_code/train/train_label.csv',
    #'guan_test_data':'dataset/qiuguan/non_code/guan_test/guan_test_data.csv',
    #'guan_test_label':'dataset/qiuguan/non_code/guan_test/guan_test_label.csv',
    #'test_data':'dataset/qiuguan/encode/encode_1000/test/test_encode_data_1000_new.csv',
    #'test_label':'dataset/qiuguan/non_code/test/test_labels.csv',
    #'title':'dataset/xiaoguan/RF/RF_for_train/train_class_9/test/test_data.csv',
    
    #'all':''
    #'title':'data/xiaoqiu_gene_5000/train/gene_5000_gene_name.csv',
    #'all':'data/xiaoqiu_gene_5000/train/gene_5000_label_name.csv'
}

#model definition
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(3300)
        self.fc1 = nn.Linear(3300, 2000)
        self.bn1= nn.BatchNorm1d(2000)
        self.fc2 = nn.Linear(2000, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.5)
    def forward(self, x):
        x=self.bn0(x)
        x = F.relu(self.drop(self.bn1(self.fc1(x))))
        x = F.relu(self.drop(self.bn2(self.fc2(x))))
        return F.softmax(self.bn3(self.fc3(x)), dim=1)
model = MLP().cuda()
#print(model)

class MLP1(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(237)
        self.fc1 = nn.Linear(237, 2000)
        self.bn1= nn.BatchNorm1d(2000)
        self.fc2 = nn.Linear(2000, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.5)
    def forward(self, x):
        x=self.bn0(x)
        x = F.relu(self.drop(self.bn1(self.fc1(x))))
        x = F.relu(self.drop(self.bn2(self.fc2(x))))
        return F.softmax(self.bn3(self.fc3(x)), dim=1)
model1 = MLP1().cuda()
print(model1)
class MLP2(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(126)
        self.fc1 = nn.Linear(126, 2000)
        self.bn1= nn.BatchNorm1d(2000)
        self.fc2 = nn.Linear(2000, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.5)
    def forward(self, x):
        x=self.bn0(x)
        x = F.relu(self.drop(self.bn1(self.fc1(x))))
        x = F.relu(self.drop(self.bn2(self.fc2(x))))
        return F.softmax(self.bn3(self.fc3(x)), dim=1)
model2 = MLP2().cuda()
#print(model2)


class MLP3(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(177)
        self.fc1 = nn.Linear(177, 2000)
        self.bn1= nn.BatchNorm1d(2000)
        self.fc2 = nn.Linear(2000, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.5)
    def forward(self, x):
        x=self.bn0(x)
        x = F.relu(self.drop(self.bn1(self.fc1(x))))
        x = F.relu(self.drop(self.bn2(self.fc2(x))))
        return F.softmax(self.bn3(self.fc3(x)), dim=1)
model3 = MLP3().cuda()
#print(model3)

class MLP4(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(130)
        self.fc1 = nn.Linear(130, 2000)
        self.bn1= nn.BatchNorm1d(2000)
        self.fc2 = nn.Linear(2000, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.5)
    def forward(self, x):
        x=self.bn0(x)
        x = F.relu(self.drop(self.bn1(self.fc1(x))))
        x = F.relu(self.drop(self.bn2(self.fc2(x))))
        return F.softmax(self.bn3(self.fc3(x)), dim=1)
model4 = MLP4().cuda()

class MLP5(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(172)
        self.fc1 = nn.Linear(172, 2000)
        self.bn1= nn.BatchNorm1d(2000)
        self.fc2 = nn.Linear(2000, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.5)
    def forward(self, x):
        x=self.bn0(x)
        x = F.relu(self.drop(self.bn1(self.fc1(x))))
        x = F.relu(self.drop(self.bn2(self.fc2(x))))
        return F.softmax(self.bn3(self.fc3(x)), dim=1)
model5 = MLP5().cuda()
import time

import torch
import torch.nn as nn
from torch.nn import functional as F







#print(model4)
import os
import time
import argparse
import numpy as np
import pandas as pd 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
#3from tensorboardX import SummaryWriter
import torch.nn.functional as F  # 激励函数的库
#import network
#import config
#import evaluate
#import data_utils
#import Trainer



def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output


def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output



class KZDatasetPredict(data.Dataset):
    """ Construct the FM pytorch dataset. """
    #def __init__(self, file,label_file, feature_map,n_class=16):
    def __init__(self, df_list):
    
       
        self.data_info = self.get_data_info(df_list)
        
        
            
        
        
        

    def __getitem__(self, index):
        # Dataset读取图片的函数
        data,label = self.data_info[index]
        #img = Image.open(img_pth).convert('RGB')
        
        return data,label

    def __len__(self):
        return len(self.data_info)
   
    
    
    def get_data_info(self,df_list):
        #解析路径
        #转为一维list存储，每一位为【图片路径，图片类别】
        labels=[]
        data_info=[]
        #print('data_info:',df_list[-1])
        #df=pd.read_csv(csv_path,sep=',')
        #df=df.iloc[:,1:]
        
        #print(df.iloc[:,-1])
        #df=df.applymap(ast.literal_eval)
        label=int(df_list[-1])
        labels.append(label)
        #print('labels:',labels)
        data=df_list[:-1]
        #df_np=np.array(df_list)
        #print(rows,cols)
        
        #print('labels:',labels)
        labels=np.array(labels)
        #print('labels.shape:',labels.shape)
        #print('labels:',labels)
        #labels=np.array(labels)
        labels=one_hot_smoothing(labels,nfm_config['n_class'])
        #print(labels)
        
           
        
        data=np.array(data)#
            
            
            
            
            
            
            
            
        data=torch.from_numpy(data)#
            
        labels=torch.from_numpy(labels)#
        #bi_data=embding_process(nfm_config,data)
        #print("bi_data.shape:",bi_data.shape)
            
            
        data_info.append((data,label))
        return data_info
class KZDatasetTest(data.Dataset):
    """ Construct the FM pytorch dataset. """
    #def __init__(self, file,label_file, feature_map,n_class=16):
    def __init__(self, csv_path):
    
       
        self.data_info = self.get_data_info(csv_path)
        
        
            
        
        
        

    def __getitem__(self, index):
        # Dataset读取图片的函数
        data, label = self.data_info[index]
        #img = Image.open(img_pth).convert('RGB')
        
        return data, label

    def __len__(self):
        return len(self.data_info)
   
    
    
    def get_data_info(self,csv_path):
        #解析路径
        #转为一维list存储，每一位为【图片路径，图片类别】
        labels=[]
        data_info=[]
        df=pd.read_csv(csv_path,sep=',')
        df=df.iloc[:,1:]
        
        #print(df.iloc[:,-1])
        #df=df.applymap(ast.literal_eval)
        rows,cols=df.shape
        print(rows,cols)
        for i in df.iloc[:,-1]:
            #print(i)
            labels.append(int(i))
        #print('labels:',labels)
        labels=np.array(labels)
        #print('labels:',labels)
        #labels=np.array(labels)
        labels=one_hot_smoothing(labels,nfm_config['n_class'])
        #print(labels)
        for i in range(rows):
            data=df.iloc[i,:-1]
            data=data.astype(float)#
            data=np.array(data)#
            
            label=labels[i]
            #print(data.shape)
            #print(label.shape)
            #label=label.tolist()
            data=torch.from_numpy(data)#
            label=torch.from_numpy(label)#
            
            
            data_info.append((data,label))
        return data_info
import torch
import torch.nn as nn
from torch.utils.data.dataset import *
from PIL import Image
from torch.nn import functional as F
import random
from sklearn.model_selection import train_test_split
import ast
import torchvision


        

class KZDataset(Dataset):
    def __init__(self, csv_path, K,n_class,ki=0, typ='train', transform=None, rand=False):
       
        self.all_data_info = self.get_data_info(csv_path)
        
        if rand:
            random.seed(1)
            random.shuffle(self.all_data_info)
        leng = len(self.all_data_info)
        every_z_len = leng // K
        if typ == 'val':
            self.data_info = self.all_data_info[every_z_len * ki : every_z_len * (ki+1)]
        elif typ == 'train':
            self.data_info = self.all_data_info[: every_z_len * ki] + self.all_data_info[every_z_len * (ki+1) :]
            
        self.transform = transform
        
        

    def __getitem__(self, index):
        # Dataset读取图片的函数
        data, label = self.data_info[index]
        #img = Image.open(img_pth).convert('RGB')
        
        return data, label

    def __len__(self):
        return len(self.data_info)
    
    
    
    def get_data_info(self,csv_path):
        #解析路径
        #转为一维list存储，每一位为【图片路径，图片类别】
        labels=[]
        data_info=[]
        df=pd.read_csv(csv_path,sep=',')
        df=df.iloc[:,1:]
        
        #print(df.iloc[:,-1])
        #df=df.applymap(ast.literal_eval)
        rows,cols=df.shape
        
        print(rows,cols)
        for i in df.iloc[:,-1]:
            #print(i)
            labels.append(int(i))
        #print('labels:',labels)
        labels=np.array(labels)
        #print('labels:',labels)
        #labels=np.array(labels)
        labels=one_hot_smoothing(labels,nfm_config['n_class'])
        #print(labels)
        for i in range(rows):
            data=df.iloc[i,:-1]
            data=data.astype(float)#
            data=np.array(data)#
            
            label=labels[i]
            #print(data.shape)
            #print(label.shape)
            #label=label.tolist()
            data=torch.from_numpy(data)#
            label=torch.from_numpy(label)#
            
            
            data_info.append((data,label))
        return data_info
    
from torch.autograd import Variable
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
#from new_nfm_network import NFM
import os
import time
import argparse
import numpy as np
import pandas as pd 
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
#import torch.backends.cudnn as cudnn
#3from tensorboardX import SummaryWriter

#import torchmetrics
            

from torchmetrics.classification import accuracy

def train_epoch(model,train_loader,batch_size,optimizer,loss_func):
    BATCH_SIZE=batch_size
    total = 0
    correct=0
    total_loss=0
    #
    model.train()
    total_train_accuracy=0  
    for batch_idx, (x, labels) in enumerate(train_loader):
            
        labels = Variable(labels)
        x = Variable(x)
            
            
        x=torch.tensor(x,dtype=torch.float)
        labels=torch.tensor(labels,dtype=torch.float)
        x, labels = x.cuda(), labels.cuda()
        labels_int=labels=torch.max(labels,1)[1]
            
            
        optimizer.zero_grad()
        y_predict = model(x)
            
        loss = loss_func(y_predict, labels)
        loss.backward()
        optimizer.step()
            
        loss = loss.item()
           

        total_loss += loss
            
            
            
        #
        '''
        train_acc_en=torchmetrics.Accuracy(task='multiclass',num_classes=9).cuda()
        batch_train_acc=train_acc_en(y_predict,labels_int)
        '''
        #batch_train_acc=torchmetrics.functional.accuracy(y_predict,labels_int)
        #batch_train_acc=torchmetrics.classification.Accuracy(y_predict,labels_int)
        batch_train_acc = torchmetrics.functional.accuracy(y_predict, labels_int, task='multiclass', num_classes=nfm_config['n_class'])
        
        
        #train_acc_en=accuracy(y_predict,labels_int)
        #batch_train_acc=train_acc_en()
        
        total_train_accuracy+=batch_train_acc
            
    total_train_accuracy/=(batch_idx+1)
    print('total_train_accuracy:',total_train_accuracy)
    print("Training Epoch: %d, total loss: %f" % (epoch_id, total_loss))
    return total_loss,total_train_accuracy

def val_epoch(model,test_loader,batch_size,optimizer): 
    batch_size_num=0
    total_test_acc=0
    model.eval()
    for i , (inputs , targets) in enumerate(test_loader):   
            print("test")
            
            inputs = Variable(inputs)   
            targets = Variable(targets)     
           
            inputs=torch.tensor(inputs ,dtype=torch.float)   
            targets=torch.tensor(targets ,dtype=torch.float)   
            inputs , targets = inputs.cuda(),  targets.cuda()   
            yhat = model(inputs)  
            
            
            
            targets=torch.max(targets,1)[1]
            
            
            
            #test_acc_en=torchmetrics.Accuracy(task='multiclass',num_classes=9).cuda()
            #batch_test_acc=torchmetrics.functional.accuracy(yhat,targets)
            batch_test_acc = torchmetrics.functional.accuracy(yhat, targets, task='multiclass', num_classes=nfm_config['n_class'])
            total_test_acc+=batch_test_acc
            
            batch_size_num=i
    total_test_acc/=(batch_size_num+1)
        ###print('total_test_accuracy:',total_test_acc/(batch_size+1))
    print('total_test_accuracy:',total_test_acc)
        
                    
                    
            
            
    
        
   
    
    return total_test_acc


import numpy as np
import math
import matplotlib.pyplot as plt

def plotLoss(loss,epoch):
    plt.rcParams['font.sans-serif']=['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    plt.figure(figsize=(8, 5))
    x=[i for i in range(epoch)]
    #acc_train=acc_train.cpu()
    #acc_test=acc_test.cpu()
    plt.plot(x, loss, 'r-', mec='k', label='Logistic Loss', lw=2)
    #plt.plot(x,acc_train,'b-',mec='k',label='accuracy Train',lw=2)
    #plt.plot(x,acc_test,'g-',mec='k',label='accuracy Test',lw=2)
    #plt.plot(x, y_01, 'g-', mec='k', label='0/1 Loss', lw=2)
    #plt.plot(x, y_hinge, 'b-',mec='k', label='Hinge Loss', lw=2)
    #plt.plot(x, boost, 'm--',mec='k', label='Adaboost Loss',lw=2)
    plt.grid(True, ls='--')
    plt.legend(loc='upper right')
    plt.title('损失函数')
    plt.show()
 


MLP1(
  (bn0): BatchNorm1d(237, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=237, out_features=2000, bias=True)
  (bn1): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=2000, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=100, out_features=9, bias=True)
  (bn3): BatchNorm1d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)


In [6]:
#find the means for the predicted correctly
from torch.autograd import Variable 
from torch.utils.data import DataLoader 
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import accuracy_score 
 
def bool_predict_model(test_dl, model): 
    model.eval()#测试数据稳定
    error_dataset=torch.tensor([0])
    error_dataset=error_dataset.view(1,1)
    predictions, actuals = [], [] 
    num=0
    for i, (inputs,targets) in enumerate(test_dl): 
        # evaluate the model on the test set 
        #print(\ inputs:\ ,inputs) 
        #print(\ targets:\ ,targets) 
        inputs = Variable(inputs) 
        #bi_inputs=Variable(bi_inputs)
        targets = Variable(targets) 
        #print(targets)
        #print('targets:',targets.shape)        
        #targets=targets.argmax(axis=1)  
        #print('targets:',targets.shape)
        #x = torch.tensor(x, dtype=torch.float) 
        #x=x.clone().detach().requires_grad_(True) 
        inputs=torch.tensor(inputs,dtype=torch.float) 
        #bi_inputs=torch.tensor(bi_inputs,dtype=torch.float)
        targets=torch.tensor(targets,dtype=torch.float) 
        inputs, targets = inputs.cuda(),targets.cuda() 
        yhat = model(inputs) 
        
        yhat=yhat.argmax(axis=1)
        #print('yhat:',yhat.shape)
        
        if targets==yhat:
            return True
        else:
            return False
#find

In [7]:
#find the samples that predicted correctly
import pandas as pd 
test_df=pd.read_csv('dataset/qiuguan/origin_800/xiaoqiu_xiaoguan/test_info1.csv',sep=',')
test_df=test_df.iloc[:,1:]
rows,cols=test_df.shape
#print(rows,cols)
import torch

#功能：加载保存到path中的各层参数到神经网络

path='models/ConvAttMLP_10layers_add//MLP310.pt'
#path='models/ConvAttMLP_res_concat/MLP410.pt'

#nfm=NFM(nfm_config)
#mlp=MLP()
model=ConvAttMLP_10layers_add()
model.cuda()
model.load_state_dict(torch.load(path), strict=False)
'''
# 输出模型的参数名和参数形状
for name, param in model.named_parameters():
    print(f"Parameter Name: {name}, Shape: {param.size()}")
'''

'''
model_lrp=ConvAttMLP_LRP()
#print(nfm)
#net = nn.DataParallel(net)
#net = net.to(device)
# Load the state_dict for both models separately
model.load_state_dict(torch.load(path), strict=False)
model_lrp.load_state_dict(model.state_dict())  # Copying the loaded state dict to model_lrp

model.cuda()
model_lrp.cuda()
print(model)




model_params = list(model.named_parameters())
#print(nfm_params)
net=model
'''

#testset = KZDatasetPredict(test_df)
''''
test_loader = data.DataLoader(
         dataset=testset,
         #transform=torchvision.transforms.ToTensor(),
         drop_last=True,
         batch_size=nfm_config['batch_size']
        
     )
'''
false_list=[]
for i in range(rows):
    df_list=test_df.iloc[i,:].tolist()
    #print(type(df_list))
    #print('df_list:',df_list[-1])
    #print(data_set)
    #print('this is data_set')
    data_set = KZDatasetPredict(df_list)
    data_test_loader=data.DataLoader(dataset=data_set,
                                    batch_size=1)
    #print(data_test_loader)
    bool_index=bool_predict_model(data_test_loader,model)
    #print(bool_index)
    if bool_index==False:
        
        false_list.append(i)
        #test_df=test_df.drop(index=i, inplace=True)
        
for i,aitem in enumerate(false_list):
    test_df.drop(index=i, inplace=True)
print(test_df.shape)   

test_df.to_csv('dataset/qiuguan/origin_800/xiaoqiu_xiaoguan/ConvAttMLP_10layers_add//test_info3.csv')

/tmp/ipykernel_26270/4283510531.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs=torch.tensor(inputs,dtype=torch.float)
/tmp/ipykernel_26270/4283510531.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets=torch.tensor(targets,dtype=torch.float)


(127, 3301)


In [8]:
#top_20


#compute mean value of all the correctly predicted samples:
import pandas as pd 
import torch
import numpy as np
test_df=pd.read_csv('dataset/qiuguan/origin_800/xiaoqiu_xiaoguan/ConvAttMLP_10layers_add//test_info3.csv',sep=',')
test_df=test_df.iloc[:,1:]
rows,cols=test_df.shape
#print(rows,cols)
columns=test_df.columns[:-1].tolist()
#columns=torch.tensor(columns).cuda()#ValueError: too many dimensions 'str'


row_mean=test_df.mean(axis=0)
#print(row_mean)
row_mean=row_mean[:-1]#drop off label
row_mean_np=np.array(row_mean).reshape(1,-1)
#print(row_mean_np)
row_mean_df=pd.DataFrame(row_mean_np)
#print(row_mean_df)
row_mean_df.columns=test_df.columns[:-1]
#print(row_mean_df)
row_mean_tensor=torch.from_numpy(row_mean_np).cuda()
row_mean_tensor=torch.tensor(row_mean_tensor,dtype=torch.float)###################################
#去掉行
#print('row_mean_tensor:',row_mean_tensor.shape)

row_mean_tensor.cuda()
#print('x_bn0:',x.shape)
#x2=torch.zeros([1,1100])
#x2=x2.cuda()
#xm=torch.cat([row_mean_tensor,x2],dim=1)

#compute contribution using the Equ.
import torch

#功能：加载保存到path中的各层参数到神经网络

import torch

#功能：加载保存到path中的各层参数到神经网络

import torch

#功能：加载保存到path中的各层参数到神经网络

import torch

#功能：加载保存到path中的各层参数到神经网络

import torch

#功能：加载保存到path中的各层参数到神经网络

path='models/ConvAttMLP_10layers_add/MLP310.pt'

#nfm=NFM(nfm_config)
model=ConvAttMLP_10layers_add()
model_lrp=ConvAttMLP_10layers_add_LRP()
#print(nfm)
#net = nn.DataParallel(net)
#net = net.to(device)
# Load the state_dict for both models separately


model.load_state_dict(torch.load(path), strict=False)
'''
# 将原始网络的参数赋值给子网络
for param_name, param in model.named_parameters():
    if hasattr(model_lrp, param_name):
        sub_param = getattr(model_lrp, param_name)
        if sub_param.data.shape == param.data.shape:
            sub_param.data = param.data.clone()
'''            
#model_lrp.named_parameters['fc22.weight']=model.named_parameters['fc2.weight'][:,:1100]
#model_lrp.named_parameters['fc22.bias']=model.named_parameters['fc2.bias']
model_lrp.fc22.weight.data=model.fc2.weight.data[:,:1100]

'''            
# 输出模型的参数名和参数形状
for name, param in model_lrp.named_parameters():
    print(f"Parameter Name: {name}, Shape: {param.size()}")
'''            
            
#model_lrp.load_state_dict(model.state_dict())  # Copying the loaded state dict to model_lrp

model.cuda()
model_lrp.cuda()











def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    n = len(labels)
    #n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output







import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline

from innvestigator2 import InnvestigateModel
#from utils import Flatten
model.double()
model.eval()
model.cuda()

inn_model = InnvestigateModel(model_lrp, lrp_exponent=2,
                              method="e-rule",
                              beta=.5)

genes_features=np.array([i for i  in range(9)])#################
genes_features=genes_features.reshape(9,1).tolist()#######################genes_features[i][0]=label

inn_model.cuda()

###[3, 25.353026075712233, tensor([ 182,  879,  103, 2657, 2489,  914, 2437,  180, 2417, 1402, 2344, 2947,
###values=tensor([0.0572, 0.0495, 0.0404, 0.0381, 0.0364, 0.0353, 0.0328, 0.0302, 0.0284,
#        0.0257, 0.0180, 0.0164, 0.0155, 0.0150, 0.0135, 0.0127, 0.0121, 0.0120,
#        0.0114, 0.0107, 0.0107, 0.0104, 0.0100, 0.0100, 0.0099, 0.0091, 0.0091,
#       0.0088, 0.0083, 0.0082, 0.0081, 0.0079, 0.0078, 0.0077, 0.0076, 0.0074,
#       0.0073, 0.0072, 0.0072, 0.0071], dtype=torch.float64),
###indices=tensor([ 182,  879,  103, 2657, 2489,  914, 2437,  180, 2417, 1402, 2344, 2947,
#        2546, 1114,  796, 1111, 2472, 2326, 1274,  932, 2476,  716,  989, 3289,
#        1252, 2053,  785, 2429, 3015, 1585,  975, 1150, 1155,  726,  823, 2303,
#         699,  349, 1792, 1524])), 25.359040192320702, tensor([2489,  826, 1753, 1792,  601, 2303, 1053,  545, 2559,  624, 3256,  762,
#        2666,  182, 1881, 1585,  726, 1367, 2405, 1171, 2947, 2093,   14,  265,
#         716,  180, 1467, 2207, 3223,  349,  277, 2141, 2878, 2427, 2326, 1111,
#         746, 1402, 2150, 1602]), torch.return_types.topk(
#values=tensor([0.0456, 0.0445, 0.0406, 0.0349, 0.0277, 0.0273, 0.0246, 0.0236, 0.0225,
#        0.0221, 0.0213, 0.0200, 0.0185, 0.0182, 0.0181, 0.0153, 0.0151, 0.0142,
##        0.0141, 0.0140, 0.0138, 0.0123, 0.0121, 0.0115, 0.0109, 0.0108, 0.0107,
#        0.0102, 0.0102, 0.0098, 0.0094, 0.0093, 0.0092, 0.0089, 0.0082, 0.0081,
#        0.0080, 0.0080, 0.0079, 0.0076], dtype=torch.float64),
#indices=tensor([2489,  826, 1753, 1792,  601, 2303, 1053,  545, 2559,  624, 3256,  762,
#        2666,  182, 1881, 1585,  726, 1367, 2405, 1171, 2947, 2093,   14,  265,
#         716,  180, 1467, 2207, 3223,  349,  277, 2141, 2878, 2427, 2326, 1111,
#         746, 1402, 2150, 1602]))]

row_mean_tensor=row_mean_tensor.cuda()
#print('batch_size:',batch_size)#=20
evidence_for_class = []
#print("target:",target.shape)
#print('target:',target[3])
# Overlay with noise 
# data[0] += 0.25 * data[0].max() * torch.Tensor(np.random.randn(28*28).reshape(1, 28, 28))
#model_prediction, true_relevance = inn_model.innvestigate(in_tensor=data)
contri_k=[]
contri_k_names=[]
for i in range(9):#10类
    # Unfortunately, we had some issue with freeing pytorch memory, therefore 
    # we need to reevaluate the model separately for every class.
        model_prediction, input_relevance_values = inn_model.innvestigate(in_tensor=row_mean_tensor)
        #model_prediction, input_relevance_values = inn_model.innvestigate(in_tensor=xm)
        evidence_for_class.append(input_relevance_values)
        top_k=torch.topk(input_relevance_values,100,largest=True)#################20
        #print('top_k:',top_k)
    
        contri_k.append(top_k)
        top_k_indices=top_k.indices.cpu().detach().numpy().tolist()
        #print('top_k_indices:',top_k_indices)
        # 将二维列表转换为一维列表
        flattened_indices = [item for sublist in top_k_indices for item in sublist]

        top_k_names=[columns[j] for j in flattened_indices]
        #print('top_k_names:',top_k_names)
        contri_k_names.append(top_k_names)
        #print('contri_k_names:',contri_k_names)
        
print('begin=============================')
l=contri_k_names[4]
#l1=contri_k_names[0]
s=set(l)
inters=[]
for i in contri_k_names:
    if i!=4:
        s1=set(i)
        inter=s-s1
        print(len(inter))
        #s1=set1.intersection(set2)
        inters.append(inter)
union_set = set().union(*inters)

# 如果需要结果为列表形式
union_list = list(union_set)
print(union_list)
print(len(union_list))
print('end=================================')
        
        
        
        
        
contri_k_names_np=np.array(contri_k_names)
#print(contri_k_names_np.shape)

contri_k_names_df=pd.DataFrame(contri_k_names_np)
contri_k_names_df.to_csv('results_genes/ConvAttMLP_10layers_add//LRP/contri_100_names_df.csv')
#from chartGPT of the following code
result = set()
for sublist in contri_k_names:
    result = result.union(sublist)

# 或者使用集合的union方法的更简洁写法，使用集合解析
# result = set().union(*list_of_lists)

# 或者使用 | 运算符
# result = set().union(*list_of_lists)

# 将结果转回列表
result_list = list(result)

#print(result_list)

import csv
csv_filename = "results_genes/ConvAttMLP_10layers_add//LRP/genes_all_100_mean.csv"

# 使用CSV模块保存列表为CSV文件
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(result_list)

#这将输出所有列表的并集，不包含重复的元素。在这个例子中，结果会是 [1, 2, 3, 4, 5, 6, 7]。你可以根据你的实际需求将这个结果转换为列表或者保留为集合，具体取决于你的应用场景。
#抽取训练集和测试集中的数据

#生成新选取元素的数据集
train_path='dataset/qiuguan/origin_800/xiaoqiu_xiaoguan/train_val_info.csv'
test_path='dataset/qiuguan/origin_800/xiaoqiu_xiaoguan/test_info.csv'

train_df=pd.read_csv(train_path,sep=',')
train_df1=train_df
#print(train_df1.columns)
train_df=train_df.iloc[:,1:]
train_df1=train_df1.iloc[:,1:-1]

columns=result_list
columns.append('label')
#print(columns)


train_df2=train_df[columns]
#print(train_df2.shape)
train_df2.to_csv('dataset/qiuguan/origin_800/xiaoqiu_xiaoguan/ConvAttMLP_10layers_add//LRP/train_val_info_100.csv')

test_df=pd.read_csv(test_path,sep=',')

test_df=test_df.iloc[:,1:]



test_df=test_df[columns]
#print(test_df.shape)
test_df.to_csv('dataset/qiuguan/origin_800/xiaoqiu_xiaoguan/ConvAttMLP_10layers_add//LRP/test_info_100.csv')





/tmp/ipykernel_26270/1989142859.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  row_mean_tensor=torch.tensor(row_mean_tensor,dtype=torch.float)###################################


RuntimeError: CUDA out of memory. Tried to allocate 28.00 MiB (GPU 0; 11.91 GiB total capacity; 68.35 MiB already allocated; 5.75 MiB free; 84.00 MiB reserved in total by PyTorch)

In [8]:
x=torch.zeros([1, 3300])
y=torch.squeeze(x,0)
print(y.shape)

torch.Size([3300])
